In [2]:
import pickle 
import pandas as pd 
import wandb

api = wandb.Api()

with open('../../data/Hazumi_features/Hazumi1911_features.pkl', mode='rb') as f:
    SS, TS, SP, TP, text, audio, visual, vid = pickle.load(f, encoding='utf-8')

def get_wandb_result(project_name):
    runs = api.runs(f"r-yanagimoto/{project_name}")

    summary_list, config_list, name_list = [], [], []
    for run in runs: 
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })

    return runs_df

In [15]:
text_neg_acc = {}
text_neg_rat = {}
text_neu_acc = {}
text_neu_rat = {}
text_pos_acc = {}
text_pos_rat = {}
audio_neg_acc = {}
audio_neg_rat = {}
audio_neu_acc = {}
audio_neu_rat = {}
audio_pos_acc = {}
audio_pos_rat = {}
visual_neg_acc = {}
visual_neg_rat = {}
visual_neu_acc = {}
visual_neu_rat = {}
visual_pos_acc = {}
visual_pos_rat = {}
runs_df = get_wandb_result("self sentiment")
for _, run in runs_df.iterrows():
    uid = run["name"]
    modal = run["config"]["modal"]
    if "acc" in run["summary"].keys():
        acc = round(run["summary"]["acc"], 3)
        if modal == 't':
            text_neg_acc[uid] = f"{round(run['summary']['neg acc'], 2)} ({run['summary']['neg s']}/{run['summary']['neg s']})"
            text_neu_acc[uid] = f"{round(run['summary']['neu acc'], 2)} ({run['summary']['neu s']}/{run['summary']['neu s']})"
            text_pos_acc[uid] = f"{round(run['summary']['pos acc'], 2)} ({run['summary']['pos s']}/{run['summary']['pos s']})"
        elif modal == 'a':
            audio_acc[uid] = acc
        else:
            visual_acc[uid] = acc
    count = run["config"]


acc = {
    "text 低群": text_neg_acc,
    "text 中群": text_neu_acc,
    "text 高群": text_pos_acc,
}

result_df = pd.DataFrame(acc)
result_df.to_csv("hoge.txt", sep="&", header=False)
# personality = {}
# for uid in vid:
#     personality[uid] = SP[uid]

# personality_df = pd.DataFrame.from_dict(personality, orient="index", columns=["E", "A", "C", "N", "O"])

# df = pd.concat([result_df, personality_df], axis=1)

In [ ]:
1911M7002&0 (10/10)&0.74 (19/19)&0.59 (76/76)
1911M7001&0 (35/35)&0.3 (46/46)&0.89 (18/18)
1911M6003&0 (39/39)&0.32 (28/28)&0.88 (43/43)
1911M6002&-1 (0/0)&-1 (4/4)&0.86 (50/50)
1911M6001&0 (12/12)&0 (27/27)&1 (55/55)
1911M5002&-1 (0/0)&-1 (29/29)&0 (48/48)
1911M5001&0 (9/9)&0.5 (28/28)&0.73 (67/67)
1911M4002&-1 (0/0)&-1 (30/30)&0.88 (77/77)
1911M4001&0 (4/4)&0.01 (79/79)&1 (12/12)
1911M2003&0 (17/17)&0.2 (50/50)&0.97 (31/31)
1911M2002&0 (32/32)&0.43 (28/28)&0.95 (41/41)
1911M2001&1 (2/2)&0 (18/18)&0 (76/76)
1911F7002&0 (6/6)&0.5 (4/4)&0.41 (87/87)
1911F6003&0 (1/1)&0.25 (4/4)&0.75 (79/79)
1911F6002&-1 (0/0)&-1 (23/23)&0.93 (61/61)
1911F6001&0 (27/27)&0.3 (46/46)&0.88 (8/8)
1911F5002&0 (45/45)&0.3 (47/47)&0.9 (21/21)
1911F5001&0 (3/3)&0.44 (9/9)&0.83 (98/98)
1911F4003&0 (11/11)&0.52 (29/29)&0.78 (50/50)
1911F4002&0 (22/22)&0.23 (75/75)&0.92 (13/13)
1911F4001&0 (23/23)&0.26 (66/66)&0.88 (8/8)
1911F3003&0 (60/60)&0.24 (17/17)&0.9 (21/21)
1911F3002&0 (42/42)&0.17 (23/23)&0.75 (12/12)
1911F3001&0 (23/23)&0 (31/31)&1 (20/20)
1911F2002&0 (44/44)&0.09 (69/69)&-1 (0/0)
1911F2001&0.06 (18/18)&0 (19/19)&0.94 (34/34)

In [90]:
import itertools # 効率的なループ実行のためのイテレータ生成関数のインポート
import matplotlib.pyplot as plt # グラフ描画ライブラリ
import seaborn as sns # データの可視化を行うライブラリ
sns.set() # デフォルトスタイルにseabornを適用
from scipy.stats import pearsonr # 科学技術計算ライブラリ
import numpy as np # 数値計算を効率的に行うための拡張モジュール

for i, j in itertools.combinations(df, 2): #組み合わせでforループを実行
    x = df.loc[:,[i]].values
    y = df.loc[:,[j]].values
    a, b = pearsonr(np.ravel(x), np.ravel(y)) # リストを整形し相関係数:aとp値:bの計算
    if 0 != b.round(10) < 0.10: # p値 < 0.10
        print("=" * 45)
        print(i + "----" + j)
        print("相関係数:", a.round(4))
        print("p値:", b.round(4))

text----audio
相関係数: 0.7407
p値: 0.0
text----visual
相関係数: 0.5213
p値: 0.0063
text----E
相関係数: 0.3683
p値: 0.0641
audio----visual
相関係数: 0.6605
p値: 0.0002
audio----N
相関係数: -0.3311
p値: 0.0985
A----O
相関係数: -0.6577
p値: 0.0003
C----N
相関係数: -0.3376
p値: 0.0917
